# MRPC数据集 - PyTorch 微调

## 数据集预处理
在Huggingface官方教程里提到，在使用pytorch的dataloader之前，我们需要做一些事情：

* 把dataset中一些不需要的列给去掉了，比如‘sentence1’，‘sentence2’等
* 把数据转换成pytorch tensors
* 修改列名 label 为 labels (实践证明不需要)

huggingface datasets贴心地准备了三个方法：**remove_columns, rename_column, set_format**

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [3]:
tokenized_datasets['train'].column_names

['attention_mask',
 'idx',
 'input_ids',
 'label',
 'sentence1',
 'sentence2',
 'token_type_ids']

In [4]:
# 删除没有用的行
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2','idx'])
# 修改列名 label 为 labels (实践证明不需要)
tokenized_datasets = tokenized_datasets.rename_column('label','labels')  
# 把数据转换为pytorch tensor
tokenized_datasets.set_format('torch')

tokenized_datasets['train'].column_names

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [5]:
# 经过上面的处理，它就可以直接丢进pytorch的Dataloader中了，跟pytorch中的Dataset格式已经一样了
tokenized_datasets['train']

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 3668
})

## pytorch dataloaders

在pytorch的DataLoader里，有一个collate_fn参数，其定义是："merges a list of samples to form a mini-batch of Tensor(s). Used when using batched loading from a map-style dataset." 我们可以直接把Huggingface的DataCollatorWithPadding对象传进去，用于对数据进行padding等一系列处理

In [6]:
from torch.utils.data import DataLoader, Dataset
# 通过这里的dataloader，每个batch的seq_len可能不同
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator)  
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator)

# 查看一下train_dataloader的元素长啥样
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

# 如果在前面数据处理过程中没有把label转化成labels 这一步也会自动完成

{'attention_mask': torch.Size([8, 96]),
 'input_ids': torch.Size([8, 96]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 96])}

## 模型

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
# 前面的batch可以直接丢进model
model(**batch)

SequenceClassifierOutput([('loss', tensor(0.7619, grad_fn=<NllLossBackward0>)),
                          ('logits', tensor([[0.2469, 0.0111],
                                   [0.2622, 0.0340],
                                   [0.2661, 0.0124],
                                   [0.2734, 0.0322],
                                   [0.2622, 0.0440],
                                   [0.2671, 0.0257],
                                   [0.2718, 0.0234],
                                   [0.2744, 0.0361]], grad_fn=<AddmmBackward0>))])

## 训练

In [9]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)  # num of batches * num of epochs
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,  # scheduler是针对optimizer的lr的
    num_warmup_steps=0,
    num_training_steps=num_training_steps)
print(num_training_steps)

1377


In [10]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### 编写pytorch training loops:
1. for每一个epoch
2. 从dataloader里取出一个个batch
3. 把batch喂给model（先把batch都移动到对应的device上）
4. 拿出loss，进行反向传播backward
5. 分别把optimizer和scheduler都更新一个step

In [11]:
from tqdm import tqdm

for epoch in range(num_epochs):
    for batch in tqdm(train_dataloader):
        # 要在GPU上训练，需要把数据集都移动到GPU上：
        batch = {k:v.to(device) for k,v in batch.items()}
        loss = model(**batch).loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

100%|██████████| 459/459 [00:37<00:00, 12.25it/s]


In [12]:
from datasets import load_metric

metric= load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():  # evaluation的时候不需要算梯度
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    # 由于dataloader是每次输出一个batch，因此我们要等着把所有batch都添加进来，再进行计算
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

{'accuracy': 0.8529411764705882, 'f1': 0.8969072164948454}